In [1]:
!ls

cloze_task_average_accuracy.xlsx  Compare_data.md	 Tag_cloze_task.ipynb
cloze_task_raw_data.xlsx	  LSTM_probability.xlsx
Compare_data.ipynb		  output.xlsx


In [1]:
import pandas as pd
data = pd.read_excel('LSTM_probability.xlsx')
data_cloze = pd.read_excel('cloze_task_raw_data.xlsx')

In [2]:
data.head()

,item.id,word.serial.no,shown,answer,prob
0,1,1,Введите первое слово,на,0.016503
1,1,2,На,болотах,0.000006
2,1,3,На болотах,оставался,0.000138
3,1,4,На болотах оставался,еще,0.124899
4,1,5,На болотах оставался ещё,лед,0.002193


In [3]:
data_cloze.shape

(64645, 16)

In [4]:
#for each sentence replace 'Введите первое слово'  with 'Start_X' according to sentence id
data_cloze.shown.replace('Введите первое слово', 'Start', inplace=True)
data_cloze[data_cloze.shown=='Start']
d_c_start = data_cloze[['shown', 'item.id']]
d_c_start['item.id'] = d_c_start['item.id'].astype(str)
d_c_start.loc[d_c_start.shown == 'Start', 'XXX'] = d_c_start.shown+'_'+d_c_start['item.id']

/home/semen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/semen/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/semen/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [5]:
data_cloze['app'] = d_c_start.XXX
import numpy as np
data_cloze['shown'] = np.where(data_cloze['shown']=='Start', data_cloze.app , data_cloze.shown)
data_cloze = data_cloze.drop('app', axis=1)

In [6]:
data_cloze.shape, len(data_cloze.shown.unique())

((64645, 16), 1309)

In [7]:
gb = data_cloze.groupby(['shown', 'answer', 'accuracy']).size()
gb = pd.DataFrame(gb,columns = ['fr'])
gb = gb.reset_index(level=['shown', 'answer', 'accuracy'])
gb.shape

(25528, 4)

In [8]:
gg= data_cloze.groupby(['shown', 'accuracy']).size()
gg = gg.reset_index(level=['shown', 'accuracy']) 
gg.columns = ['shown', 'accuracy', 'all_fr']
part0 = gg[gg.all_fr.max() == gg.all_fr]
max_value = gg[gg.shown == 'В'].all_fr.sum()
print(max_value)
gg.head(), gg.shape

701


(       shown  accuracy  all_fr
 0    Start_1       0.0     138
 1   Start_10       0.0      70
 2   Start_10       1.0       1
 3  Start_100       0.0      16
 4  Start_101       0.0      16, (1978, 3))

In [9]:
gk= gg[['shown', 'all_fr', 'accuracy']].groupby(['shown']).sum()
gk = gk.reset_index(level=['shown'])
gk.columns = ['shown', 'sum_fr', 'accuracy']
gk.head(), gk.shape
gk[gk.accuracy==1.].shape

(679, 3)

In [10]:
gg = gg.merge(gk, left_on='shown', right_on='shown', how='outer')


In [11]:
gg.head()

,shown,accuracy_x,all_fr,sum_fr,accuracy_y
0,Start_1,0.0,138,138,0.0
1,Start_10,0.0,70,71,1.0
2,Start_10,1.0,1,71,1.0
3,Start_100,0.0,16,16,0.0
4,Start_101,0.0,16,16,0.0


In [12]:
print(gg.head())
def normal(x):
    return max_value/x
gg['index_norm'] = gg.sum_fr
gg.index_norm = gg.index_norm.apply(normal)
print(gg.head())
gg['prob'] = np.where(gg.accuracy_x==1., (gg.all_fr*gg.index_norm)/(gg.index_norm*gg.sum_fr), 0.0000000000000000001)
res = gg[['shown', 'accuracy_x','sum_fr','prob']]
res = res[res.prob!=0.]
print(res.shape)
print(res.head())

       shown  accuracy_x  all_fr  sum_fr  accuracy_y
0    Start_1         0.0     138     138         0.0
1   Start_10         0.0      70      71         1.0
2   Start_10         1.0       1      71         1.0
3  Start_100         0.0      16      16         0.0
4  Start_101         0.0      16      16         0.0
       shown  accuracy_x  all_fr  sum_fr  accuracy_y  index_norm
0    Start_1         0.0     138     138         0.0    5.079710
1   Start_10         0.0      70      71         1.0    9.873239
2   Start_10         1.0       1      71         1.0    9.873239
3  Start_100         0.0      16      16         0.0   43.812500
4  Start_101         0.0      16      16         0.0   43.812500
(1978, 4)
       shown  accuracy_x  sum_fr          prob
0    Start_1         0.0     138  1.000000e-19
1   Start_10         0.0      71  1.000000e-19
2   Start_10         1.0      71  1.408451e-02
3  Start_100         0.0      16  1.000000e-19
4  Start_101         0.0      16  1.000000e-19


In [13]:
r = res[['shown','accuracy_x', 'prob']].groupby(['shown']).sum()
r = r.reset_index(level=['shown'])
print(r.shape)
#r.sum_fr = np.where(r.shown==res.shown, res.sum_fr, 0)
r.head()

(1309, 3)


,shown,accuracy_x,prob
0,Start_1,0.0,1.000000e-19
1,Start_10,1.0,1.408451e-02
2,Start_100,0.0,1.000000e-19
3,Start_101,0.0,1.000000e-19
4,Start_102,0.0,1.000000e-19


In [14]:
r[r.prob>=1.]

,shown,accuracy_x,prob
233,В современном обществе семья и школа оказывают...,1.0,1.0
305,Во избежание ожогов надо нанести на лицо небол...,1.0,1.0
307,Во избежание ожогов надо нанести на лицо небол...,1.0,1.0
389,Дрозды и скворцы начали вить семейные гнёзда н...,1.0,1.0
511,Зачем ему звонить‚ если откликается спокойный ...,1.0,1.0
550,Ирине досталась отдельная комната в двухкомнатной,1.0,1.0
565,Какие главные лекарства должны входить,1.0,1.0
802,Олень бродил среди берёз‚ жевал талый снег,1.0,1.0
1022,С нескрываемой едкой иронией отзываются они др...,1.0,1.0
1040,Собаку‚ виновницу случившегося‚ приказали сечь...,1.0,1.0


In [15]:
data.shown.unique().shape

(1170,)

In [16]:
data.shown.replace('Введите первое слово', 'Start', inplace=True)
#data[data.shown=='Start']
d_start = data[['shown', 'item.id']]
d_start['item.id'] = d_start['item.id'].astype(str)
d_start.loc[d_start.shown == 'Start', 'XXX'] = d_start.shown+'_'+d_c_start['item.id']
data['app'] = d_start.XXX
data['shown'] = np.where(data['shown']=='Start', data.app , data.shown)
data = data.drop('app', axis=1)
data.shown.unique().shape


/home/semen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/semen/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/semen/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

(1185,)

In [17]:
data_m = data.merge(r, left_on='shown', right_on='shown', how='outer')

In [18]:
data_m.shown.unique().shape

(1310,)

In [19]:
#data_m = data_m.dropna()
result = data_m[['shown', 'answer', 'prob_x', 'prob_y']]
result[result.prob_y<0.0001]

,shown,answer,prob_x,prob_y
0,Start_102,на,1.650273e-02,1.000000e-19
1,Start_102,он,1.442693e-02,1.000000e-19
2,Start_102,ваня,3.091045e-05,1.000000e-19
3,Start_102,сделав,3.943548e-05,1.000000e-19
4,Start_102,я,1.338693e-02,1.000000e-19
5,Start_102,дорога,6.815995e-05,1.000000e-19
6,Start_102,не,7.830556e-03,1.000000e-19
7,Start_102,выходя,3.651174e-05,1.000000e-19
8,Start_102,и,3.465874e-02,1.000000e-19
9,Start_102,у,5.279777e-03,1.000000e-19


In [21]:
#print(check.shape)
check = data_cloze[['shown', 'answer']]
check = check.drop_duplicates()
data_result =  result.merge(check, left_on='shown', right_on='shown', how='outer')
data_result.shape

(38346, 5)

In [22]:
data_result = data_result[data_result.answer_x == data_result.answer_y]
data_result = data_result[['shown','answer_x','prob_x','prob_y']]

In [23]:
data_result.columns = ['shown', 'answer', 'prob_by_model', 'prob_by_human']
data_result['$R^2$'] = (data_result.prob_by_human-data_result.prob_by_model)**2
data

,item.id,word.serial.no,shown,answer,prob
0,1,1,Start_102,на,0.016503
1,1,2,На,болотах,0.000006
2,1,3,На болотах,оставался,0.000138
3,1,4,На болотах оставался,еще,0.124899
4,1,5,На болотах оставался ещё,лед,0.002193
5,1,6,На болотах оставался ещё лёд‚,но,0.039917
6,1,7,На болотах оставался ещё лёд‚ но,на,0.031502
7,1,8,На болотах оставался ещё лёд‚ но на,берегах,0.003701
8,1,9,На болотах оставался ещё лёд‚ но на берегах,реки,0.014212
9,1,10,На болотах оставался ещё лёд‚ но на берегах реки,появилась,0.000670


In [24]:
data_result[data_result.prob_by_human<0.0001]

,shown,answer,prob_by_model,prob_by_human,$R^2$
24,Start_102,он,0.014427,1.000000e-19,2.081363e-04
79,Start_102,я,0.013387,1.000000e-19,1.792099e-04
101,Start_102,не,0.007831,1.000000e-19,6.131761e-05
226,Start_102,город,0.000138,1.000000e-19,1.910112e-08
241,Start_102,в,0.063573,1.000000e-19,4.041504e-03
10456,Start_50,однако,0.006184,1.000000e-19,3.823848e-05
10573,Start_50,он,0.014427,1.000000e-19,2.081363e-04
11037,Start_50,он,0.014427,1.000000e-19,2.081363e-04
11165,Start_50,у,0.005280,1.000000e-19,2.787604e-05
11629,Start_50,у,0.005280,1.000000e-19,2.787604e-05


In [25]:
measure = data_result[data_result.prob_by_model<data_result.prob_by_human].shape[0]/data_result.shape[0]
print("{}% of data, where prob of cloze task is greater than prob of lstm ".format(round(measure,2)))

0.81% of data, where prob of cloze task is greater than prob of lstm 


In [26]:
data_result['Cross_entropy'] = -(0.0*np.log(data_result.prob_by_model)+1.0*np.log(data_result.prob_by_human))

/home/semen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [27]:
data_result.head()

,shown,answer,prob_by_model,prob_by_human,$R^2$,Cross_entropy
24,Start_102,он,0.014427,1.000000e-19,2.081363e-04,43.749117
79,Start_102,я,0.013387,1.000000e-19,1.792099e-04,43.749117
101,Start_102,не,0.007831,1.000000e-19,6.131761e-05,43.749117
226,Start_102,город,0.000138,1.000000e-19,1.910112e-08,43.749117
241,Start_102,в,0.063573,1.000000e-19,4.041504e-03,43.749117


In [31]:
data_result[data_result.prob_by_human<0.00000000001]

,shown,answer,prob_by_model,prob_by_human,$R^2$,Cross_entropy
24,Start_102,он,0.014427,1.000000e-19,2.081363e-04,43.749117
79,Start_102,я,0.013387,1.000000e-19,1.792099e-04,43.749117
101,Start_102,не,0.007831,1.000000e-19,6.131761e-05,43.749117
226,Start_102,город,0.000138,1.000000e-19,1.910112e-08,43.749117
241,Start_102,в,0.063573,1.000000e-19,4.041504e-03,43.749117
10456,Start_50,однако,0.006184,1.000000e-19,3.823848e-05,43.749117
10573,Start_50,он,0.014427,1.000000e-19,2.081363e-04,43.749117
11037,Start_50,он,0.014427,1.000000e-19,2.081363e-04,43.749117
11165,Start_50,у,0.005280,1.000000e-19,2.787604e-05,43.749117
11629,Start_50,у,0.005280,1.000000e-19,2.787604e-05,43.749117
